In [ ]:
import time
import sys
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import datetime 
import os

In [ ]:
def open_more_vacancies(list_vacancies, wd):
    body = wd.find_element(By.CSS_SELECTOR, 'body')
    body.send_keys(Keys.END)
    time.sleep(2)
    list_inst = wd.find_element(By.CLASS_NAME, "jobs-search__results-list")
    if len(list_inst.find_elements(By.TAG_NAME, 'li')) == len(list_vacancies):
        try:
            wd.find_element(By.CLASS_NAME, 'infinite-scroller__show-more-button').click()
        except:
            for i in range(5):
                body.send_keys(Keys.HOME)
                time.sleep(0.5)
                body.send_keys(Keys.END)
                time.sleep(0.5)
        time.sleep(2)
        if len(list_inst.find_elements(By.TAG_NAME, 'li')) == len(list_vacancies):
            print('End of the list reached')
    list_vacancies = list_inst.find_elements(By.TAG_NAME, 'li')
    return list_vacancies

In [ ]:
# # all
# urls = {
#     'ML London': 'https://www.linkedin.com/jobs/search/?currentJobId=3514645943&distance=25&f_WT=1%2C3&geoId=102257491&keywords=machine%20learning',
#     'DS London': 'https://www.linkedin.com/jobs/search/?currentJobId=3514645943&distance=25&f_WT=1%2C3&geoId=102257491&keywords=data%20scientist',
#     'DS worldwide': 'https://www.linkedin.com/jobs/search/?currentJobId=3635233388&f_WT=2&geoId=92000000&keywords=data%20scientist',
#     'ML worldwide': 'https://www.linkedin.com/jobs/search/?currentJobId=3596563476&f_WT=2&geoId=92000000&keywords=machine%20learning'
# }

# last 24 hours
urls = {
    'ML London': 'https://www.linkedin.com/jobs/search/?currentJobId=3656735925&distance=25&f_TPR=r86400&f_WT=1%2C3&geoId=102257491&keywords=machine%20learning',
    'DS London': 'https://www.linkedin.com/jobs/search/?currentJobId=3656735925&distance=25&f_TPR=r86400&f_WT=1%2C3&geoId=102257491&keywords=data%20scientist',
    'DS worldwide': 'https://www.linkedin.com/jobs/search/?currentJobId=3658366250&f_TPR=r86400&f_WT=2&geoId=92000000&keywords=data%20scientist',
    'ML worldwide': 'https://www.linkedin.com/jobs/search/?currentJobId=3656738385&f_TPR=r86400&f_WT=2&geoId=92000000&keywords=machine%20learning'
}

# # last 7 days
# urls = {
#     'ML London': 'https://www.linkedin.com/jobs/search/?currentJobId=3654852999&distance=25&f_TPR=r604800&f_WT=1%2C3&geoId=102257491&keywords=machine%20learning',
#     'DS London': 'https://www.linkedin.com/jobs/search/?currentJobId=3656024772&distance=25&f_TPR=r604800&f_WT=1%2C3&geoId=102257491&keywords=data%20scientist',
#     'DS worldwide': 'https://www.linkedin.com/jobs/search/?currentJobId=3655100198&f_TPR=r604800&f_WT=2&geoId=92000000&keywords=data%20scientist',
#     'ML worldwide': 'https://www.linkedin.com/jobs/search/?currentJobId=3655100198&f_TPR=r604800&f_WT=2&geoId=92000000&keywords=machine%20learning'
# }
filename = 'vacancies.csv'

In [ ]:
service = Service('D:\chromedriver\chromedriver.exe')
options = Options()
options.add_argument("--start-maximized")
wd = webdriver.Chrome(service=service, options=options)

In [ ]:
while True:
    for url_key in urls.keys():
        try:
            print(url_key)
            url = urls[url_key]

            cont = wd.get(url)

            # turn off cookies
            time.sleep(5)
            try:
                wd.find_element(By.XPATH, "//button[@action-type='DENY']").click()
                time.sleep(5)
            except:
                pass

            max_ind = 0
            if filename in os.listdir('.'):
                df = pd.read_csv(filename, index_col=0)
                max_ind = df.index.max() + 1
            else:
                df = pd.DataFrame()
            i = 0
            list_inst = wd.find_element(By.CLASS_NAME, "jobs-search__results-list")
            list_vacancies = list_inst.find_elements(By.TAG_NAME, 'li')
            try:
                while len(list_vacancies) > i:
                    if len(list_vacancies) - i < 2:
                        list_vacancies = open_more_vacancies(list_vacancies, wd)

                    try:
                        wd.find_element(By.CLASS_NAME, 'cta-modal__dismiss-btn').click()
                    except:
                        pass
                    vacancy = list_vacancies[i]

                    df.loc[max_ind + i, 'title'] = vacancy.find_element(By.CLASS_NAME, "base-search-card__title").text
                    df.loc[max_ind + i, 'company_name'] = vacancy.find_element(By.CLASS_NAME, "base-search-card__subtitle").text
                    df.loc[max_ind + i, 'location'] = vacancy.find_element(By.CLASS_NAME, "job-search-card__location").text
                    try:
                        df.loc[max_ind + i, 'salary'] = vacancy.find_element(By.CLASS_NAME, "job-search-card__salary-info").text
                    except:
                        pass
                    try:
                        df.loc[max_ind + i, 'date'] = vacancy.find_element(By.CLASS_NAME, "job-search-card__listdate"
                                                                          ).get_attribute('datetime')
                    except:
                        df.loc[max_ind + i, 'date'] = vacancy.find_element(By.CLASS_NAME, "job-search-card__listdate--new"
                                                                          ).get_attribute('datetime')
                    link = None
                    try:
                        link = vacancy.find_element(By.CLASS_NAME, 'base-card__full-link').get_attribute('href')
                    except:
                        pass

                    df.loc[max_ind + i, 'link'] = link

                    same_indexes = (df[['title', 'company_name', 'location', 'date']] == 
                                    df.loc[max_ind + i, ['title', 'company_name', 'location', 'date']]).all(axis=1)
                    if same_indexes.sum() > 1:
                        df = df.drop(index=max_ind + i)
                        if link is not None:
                            df.loc[same_indexes, 'link'] = link
                        i += 1
                        time.sleep(0.3)
                        continue
                    vacancy.click()

                    top_card_title = None
                    top_card_company = None
                    time_start = datetime.datetime.now()
                    while not ((top_card_title == df.loc[max_ind + i, 'title']) and (top_card_company == df.loc[max_ind + i, 'company_name'])):
                        time.sleep(0.5)
                        try:
                            top_card_title = wd.find_element(By.CLASS_NAME, "top-card-layout__title").text
                            top_card_company = wd.find_element(By.CLASS_NAME, "topcard__flavor").text
                        except:
                            pass

                        if (datetime.datetime.now() - time_start).seconds > 5:
                            wd.refresh()
                            time.sleep(0.5)
                            list_vacancies = wd.find_element(By.CLASS_NAME, "jobs-search__results-list").find_elements(By.TAG_NAME, 'li')
                            while len(list_vacancies) - i < 2:
                                list_vacancies = open_more_vacancies(list_vacancies, wd)
                                time.sleep(1)
                            time_start = datetime.datetime.now()
                        time.sleep(0.5)

                    try:
                        wd.find_element(By.XPATH, "//button[@data-tracking-control-name='public_jobs_show-more-html-btn']").click()
                        df.loc[max_ind + i, 'text'] = wd.find_element(By.CLASS_NAME, "show-more-less-html__markup").text
                    except:
                        pass

                    i += 1
                    time.sleep(1)

            finally:
                df[~df['text'].isna()].drop_duplicates().reset_index(drop=True).to_csv(filename)
                print('Number of vacancies were viewed:', i)
        except KeyboardInterrupt:
            raise Exception('You have pressed ctrl-c button.')
            sys.exit()
        else:
            pass